In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 
import math

In [2]:
init = [3,4,8]

In [5]:
N = len(init)
BigM = sum(init)

model = AbstractModel()
model.i = RangeSet(N)
model.j = Set(initialize = model.i)
model.t = RangeSet(8)
model.U = Var(model.i,model.j, model.t, initialize = 0, within=Binary)
model.move = Var(model.i,model.j, model.t, initialize = 0, bounds=(0,BigM/2), within=NonNegativeReals)
model.v = Var(model.i,model.t, initialize = 0, bounds=(0,BigM), within=NonNegativeReals)
model.P = Var(model.i,model.t, initialize = 0, within=Binary)
model.z = Var(model.t, initialize = 0, bounds=(0,1), within=NonNegativeReals)

def Rule_C1(model,i,t):
    if t>1:
        return model.v[i,t]== model.v[i,t-1]- sum(model.move[i,j,t-1]-model.move[j,i,t-1] for j in model.j if i!=j)
    else:
        return model.v[i,t]==init[i-1]
model.C1=Constraint(model.i, model.t, rule=Rule_C1)

def Rule_C3(model,i,j,t):
    if i!=j:
        return model.move[i,j,t] <= BigM * model.U[i,j,t]
    else:
        return Constraint.Skip 
model.C3=Constraint(model.i, model.j, model.t, rule=Rule_C3)

def Rule_C4A(model,i,t):
    if t>1:
        return -(model.v[i,t]-2*model.v[i,t-1]) <= BigM * (1- sum(model.U[j,i,t-1] for j in model.j if i!=j) )
    else:
        return Constraint.Skip
model.C4A=Constraint(model.i, model.t, rule=Rule_C4A)

def Rule_C4B(model,i,t):
    if t>1:
        return model.v[i,t]-2*model.v[i,t-1] <= BigM * (1- sum(model.U[j,i,t-1] for j in model.j if i!=j) )
    else:
        return Constraint.Skip
model.C4B=Constraint(model.i, model.t, rule=Rule_C4B)

def Rule_C5(model,t):
    return sum(model.U[i,j,t] for i in model.i for j in model.j if i!=j) <= model.z[t]
model.C5=Constraint(model.t, rule=Rule_C5)

def Rule_C6(model,i,t):
    return model.v[i,t] <= BigM * (1- model.P[i,t])
model.C6=Constraint(model.i,model.t, rule=Rule_C6)

def Rule_C7(model):
    return sum(model.P[i,t] for i in model.i for t in model.t ) >= 1
model.C7=Constraint( rule=Rule_C7)

def Rule_C8(model,i,t):
    return model.z[t]<= 1-model.P[i,t] 
model.C8=Constraint(model.i,model.t, rule=Rule_C8)

def Rule_C9(model,t):
    if t>1:
        return model.z[t]<=model.z[t-1]
    else:
        return Constraint.Skip
model.C9=Constraint(model.t, rule=Rule_C9)

def rule_OF(model):
    return sum(t*model.z[t] for t in model.t)
model.obj = Objective(rule=rule_OF, sense=minimize)

In [6]:
opt = SolverFactory('cbc')
instance = model.create_instance()  

In [7]:
results = opt.solve(instance, options={"threads": 8, 'rel_gap':0.1})
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  results.solver.status)
print(value(instance.obj))

feasible
10.0
